# Speaker Identification with Ey and Breath
> Summary
This project explores the possibilities and the methodologies to identify speakers' identities from short recordings of *`Ey` sounds* and *`breath`es*. We construct two neural networks to capture the features of sampled recordings and to predict the speaker identities. One is `conv_net + lstm`, and the other is `conv_net + tdnn`.
===

## Goals
Our goals are
1. extract features from recordings;
2. find ways to predict speaker's identity using these features, and prove their effectiveness.

## Backgrounds

## Methods
Looking at the `spectrogram` of the recordings, we observe that

[comment]: # ([pic_spectrogram]: ./drafts/images/spectrogram.jpg "Spectrogram")
1. different speakers have different harmonics and patterns;
2. a speaker having a higher pitch would have similar patterns with a different person in `log-spectrogram`;
3. the spectrograms of a same person with the same sounds have varied lengths (time durations).

In order to account for the scale invariance and the shift invariance, we propose
1. use `constant Q transform` to extract spectrograms from recordings;
2. use `convolution network` to extract features along the frequencies;
3. use `recurrent network` (Long Short-Term Memory in this task) / `Time Delay Neural Network` (TDNN) to capture the dynamics along the time steps.

### Constant Q transform
It is closely relate to the Fourier transfrom. It has series of logarithmically spaced filters, making it suitable for analysing music signals. The key components of constant Q transform are
1. The center frequencies are $f_k = 2^{\frac{k}{b}}f_0$, where $b$ is the number of filters per oactave, $k = 1, \ldots, K$, and the total number of frequency bins $K = b\log_2(\frac{f_{max}}{f_0})$.
2. The bandwidth of the $k$-th filter is $\delta f_k = f_{k+1} - f_{k} = f_k (2^{\frac{1}{b}} - 1)$. The ratio of frequency to bandwidth is constant $Q = \frac{f_k}{\delta f_k} = \frac{1}{(2^{\frac{1}{b}} - 1)}$, and hence the notation `constant Q`.
3. The window length for the $k$-th frequency bin is $N_k = Q\frac{f_s}{f_k}$, where $f_s$ is the sampling frequency.
4. Finally, the constant Q transform
$$ x[k] = \frac{1}{N_k}\sum_{n=0}^{N_k-1}{x[n]w_{N_k}[n]}e^{-j2\pi Qn/N_k}$$

### Convolution Neural Nets
1. Some patterns are much smaller than the whole image --> small filters, multi-filters
2. The same patterns appear in different regions --> shared filter
3. Subsampling the pixels will not change the object --> pooling

### Long Short-Term Memory
The LSTM is a variant of Recurrent Neural Networks (RNN). It itself has many variants too. It was originally proposed by `Sepp Hochreiter and Jürgen Schmidhuber` in 1997, and then is used to effectively model contextual sequences. It is designed with internal memory cells and controlling gates to selectively memorize states and fight vanishing or exploding gradients. See the structure of a LSTM cell,![lstm_cell](./drafts/images/lstm_cell.png) and a sequence of cell units ![lstm_seq](./drafts/images/lstm_seq.png)

####  Forward propagation
The forward pass
at time $t$
1. Forget gate: $f_t = \sigma(W_f x_t + U_f h_{t-1} + b_f) = \sigma(\hat{f}^t)$
2. Input gate: $i_t = \sigma(W_i x_t + U_i h_{t-1} + b_i) = \sigma(\hat{i}^t)$
and candidate input: $a_t = tanh(W_c x_t + U_c h_{t-1} + b_c) = tanh(\hat{a}^t)$
3. Cell state update: $C_t = f_t C_{t-1} + i_t a_t$
4. Output gate: $o_t = \sigma(W_o x_t + U_o h_{t-1} + b_o) = \sigma(\hat{o}^t)$

Let $z^t = [\hat{a}^t\, \hat{i}^t\, \hat{f}^t\, \hat{o}^t]$
5. Hidden output: $h_t = o_t tanh(C_t)$

#### Backpropagation through time
The error is backpropagated and the gradients are calculated to update the weights. Different with multilayer perception networks, we need to backpropagate the gradients through time (BPTT). ![bptt](./drafts/images/bptt.png) Using the chain rule, we unroll the network at each time step and compute the partial derivatives of the error wrt each paramter participated in. 
1. $$\frac{\partial E}{\partial o_i^t} = \frac{\partial E}{\partial h_i^t}\frac{\partial h_i^t}{o_i^t} = \delta h_i^t tanh(c_i^t)$$
$$\delta o^t = \delta h^t tanh(c^t)$$
2. $$\frac{\partial E}{\partial c_i^t} = \frac{\partial E}{\partial h_i^t}\frac{\partial h_i^t}{c_i^t} = \delta h_i^t o_i^t (1 - tanh^2(c_i^t))$$
$$\delta c^t += \delta h^t o^t (1 - tanh^2(c^t))$$
3. $$\frac{\partial E}{\partial i_i^t} = \frac{\partial E}{\partial c_i^t}\frac{\partial c_i^t}{i_i^t} = \delta c_i^t a_i^t$$
$$\delta i^t = \delta c^t a^t$$
4. $$\frac{\partial E}{\partial f_i^t} = \frac{\partial E}{\partial c_i^t}\frac{\partial c_i^t}{f_i^t} = \delta c_i^t c_i^{t-1}$$
$$\delta f^t = \delta c^t c^{t-1}$$
5. $$\frac{\partial E}{\partial a_i^t} = \frac{\partial E}{\partial c_i^t}\frac{\partial c_i^t}{a_i^t} = \delta c_i^t i_i^t$$
$$\delta a^t = \delta c^t i^t$$
6. $$\frac{\partial E}{\partial c_i^{t-1}} = \frac{\partial E}{\partial c_i^t}\frac{\partial c_i^t}{c_i^{t-1}} = \delta c_i^t f_i^t$$
$$\delta c^{t-1} = \delta c^t f^t$$
7. $$\delta \hat{a}^t = \delta a^t (1 - tanh^2(\hat{a}^t))$$
$$\delta \hat{i}^t = \delta i^t i^t (1 - i^t)$$
$$\delta \hat{f}^t = \delta f^t f^t (1 - f^t)$$
$$\delta \hat{o}^t = \delta o^t o^t (1 - o^t)$$
$$\delta z^t = [\delta \hat{a}^t\, \delta \hat{i}^t\, \delta \hat{f}^t\, \delta \hat{o}^t]$$
8. $$\delta I^t = W \delta z^t,$$ where $I^t = [x^t\, h^{t-1}]$.
$$\delta W^t = \delta z^t I^t$$

Given $x = [x_1, \ldots, x_T]$, $\delta W = \sum_{t=1}^{T} \delta W^t$.

> Truncated BPTT

#### Learning Curve of LSTM
Hard to train!
![train_lstm](./drafts/images/train_lstm.png)

### Time Delay Neural Network
It works on sequential data to recognise features independent of time-shift (i.e. sequence position, shift-invariant). The input signal is augmented with delayed copies as other inputs.

> Works like windows / tapers
> TDNN: FIR filter $x[t] = \sum_{i} \beta_i u[t-i]$ Moving average model
> RNN: IIR filter $x[t] = u[t] + \sum_{i} \alpha_i x[t-i]$ Autoregressive model
> ![FIR_IIR](./drafts/images/fir_iir.png)

Different delay mechanisms:
1. Delay lines - short memory
2. Decay - long memory
3. Gamma - long-short memory
4. ...

## Implementation
### Data Preparation
#### Feature extraction
* num_bin_per_octave = 48
* fs = 44100 Hz
* fmin = 27.5 Hz
* fmax = fs/2

#### Data Statistics
We selected from the dataset the portion in which each speaker has more than **100** samples. For the `Ey` data, this contains **12942** instances out of a total **25368** instances, corresponding to **53** speakers out of a total **805** speakers. For the `Breath` data, this contains **9376** instances out of a total **19613** instances, corresponding to **44** speakers out of a total **725** speakers. Then, for each speaker, we select 70% as training set, 20% as validation set, and 10% as test set. The datasets are the shuffled. The codes are shown below.

> Code
```python
import eybreath_data_prepare
```

### Network Structure
The structures of `conv_net + lstm` and `conv_net + tdnn`
![lstm](./drafts/images/lstm.png)
![lstm](./drafts/images/tdnn.png)


> ** Revision of structures **
>
> LSTM:
> 1. use smaller filers in first convolution layer
> 2. remove meanpooling, only take the output of the last timestep
>
> TDNN: add meanpooling after second convolution layer

> Code
```python
import w_net
class LSTMBuilder()
class ConvolutionBuilder()
class DenseBuilder()
class ActivationBuilder()
class ReshapeBuilder()
class PoolBuilder()
```

### Training
#### Loss
softmax outputs probability simplex ->

categorical cross entropy $H(p,q) = E_p[-\log q] = H(p) + D_{KL}(p||q)$ ->

KL distance (neglecting constant entropy $H(p) = E_p[-log p]$), which is actually the distance between the expected values of two negative log-likelihoods measured on the true prob $p$, in a nonparametric way. Can also be viewed in a parametric way, we want a high power (likelihood) over the hypothesized parameters. ->

minimize the expected negative log-likelihood (risk)

#### Optimizers
Let

$$g_t = \nabla_\theta J( \theta_t )$$
$$E[g^2]_t = \gamma E[g^2]_{t-1} + (1 - \gamma) g_t^2$$
$$RMS[g]_{t} = \sqrt{E[g^2]_t + \epsilon}$$

1. Stochastic gradient descent
$$\theta_{t+1} = \theta_t - \eta \nabla_\theta J( \theta)$$

2. Momentum
$$v_t = \gamma v_{t-1} + \eta \nabla_\theta J( \theta_t)$$
$$\theta_{t+1} = \theta_t - v_t$$

3. Nesterov accelerated gradient
$$v_t = \gamma v_{t-1} + \eta \nabla_\theta J( \theta_t )$$
$$\theta_{t+1} = \theta_t - (1 + \gamma) v_t  + \gamma v_{t-1}$$

4. Adagrad
$$\theta_{t+1} = \theta_t - \dfrac{\eta}{\sqrt{G_{t} + \epsilon}}  \nabla_\theta J( \theta),$$
where $G_t \in \mathbb{R}^{d \times d}$ is a diagonal matrix where each diagonal element $G_{t,ii}$ is the sum of the squares of the gradients w.r.t. $\theta_i$ up to time step $t$.

5. Adadelta

Adadelta is a gradient descent based learning algorithm that adapts the learning rate per parameter over time. It was proposed as an improvement over Adagrad, which is more sensitive to hyperparameters and may decrease the learning rate too aggressively.

$$\theta_{t+1} = \theta_t - \dfrac{RMS[\Delta \theta]_{t-1}}{RMS[g]_{t}} \nabla_\theta J( \theta_t )$$

6. RMSProp

RMSProp is a gradient-based optimization algorithm. It is similar to Adagrad, but introduces an additional decay term to counteract Adagrad’s rapid decrease in learning rate.

$$\theta_{t+1} = \theta_t - \dfrac{\eta}{RMS[g]_{t}}f_t$$

7. Adam

$$m_t = \beta_1 m_{t-1} + (1 - \beta_1) g_t,\quad \hat{m}_t = \dfrac{m_t}{1 - \beta^t_1}$$
$$v_t = \beta_2 v_{t-1} + (1 - \beta_2) g_t^2,\quad  \hat{v}_t = \dfrac{v_t}{1 - \beta^t_2}$$
$$\theta_{t+1} = \theta_{t} - \dfrac{\eta}{\sqrt{\hat{v}_t} + \epsilon} \hat{m}_t$$

#### Training specs
* Batch size: 1
* $\eta = 1e^{-4}$, $\gamma = 0.9$, and $\epsilon = 10e^{-8}$.

> Code
```python
# Optimizer
import w_net
def sgd()
def adadelta()
def rmsprop()
# Training procedures
import conv_tdnn_v0.2
def build_model()
def train_model()
```

### Implementation Tricks
1. Weights Initialization: The weights are initialized orthogonally.

> Code
```python
import w_net
def ortho_weight()
```

2. Parallel & GPU
3. Batch

## Results
### Accuracy
(Run # epoches)

| Network | CONV + LSTM | CONV + TDNN |
| ------- |------------:| -----------:|
| Ey      |  81.1299%   |   35.12%    |
| Breath  |  86.6417%   |   30.90%    |

### Parameter size v.s. Performance
1. Structure: LSTM
2. Data: Ey

| Parameter size | Err      |
|:---------------|---------:|
|35K             |~60%      |
|50K~55K         |65~70%    |
|45~50M          |20%       |
|55M             |25%       |
|75M             |Overfit   |

3. Data: Breath

| Parameter size | Err  |
|:---------------|-----:|
|45~50M          |15%   |
|55M             |25%   |


# TODO Session
~~1. run breath~~

~~2. finish tdnn~~

~~* save & reload model~~

~~3. examine code~~

* save results: errs, epochs, configs, optimizers...

* and load and show training info 

* Unseen speakers: put on dev and test sets

~~* move layers to a seperate file~~

~~4. tune: structure, params, optimizers~~

* draw logic of codes

~~5. grant git access~~

6. look into literature

7. write-up

8. setup Server

9. Momentum, NAG, Adagad, Adam

* weight: regularization

~~* dropout & weight decay~~

~~* visualize ey and breath~~

~~* data normalization~~

## Discussion & Thoughts 

> **What lies in the HEART of Machine Learning**
>
> Learning theory: Probably Approximately Correct (PAC) learning framework
> 1. Measuring the goodness of learning algorithm: loss, risk, Bayes risk, emprical risk
> 2. Functional space and its complexity: Estimation error, Approximation error, structural risk minimization, VC theory
> 3. Optimization

1. Dynamic network structure

Why dropout or dropconnect? Useful for dynamic structure, but not good enough

2. What's wrong with ReLU
3. Why gradient? Consider function and gradient in measure space
4. Visualize network topology

If we can add a back-trace pointer to trace back the strong activations

#### Update since 1/17 meeting

0. Continue tuning.

1. Use max pool.

    - Tried max pooling with size 2 along the frequencies. No improvement.
    - Why?

2. Put rest speakers in dev and test.

    - Accuracy goes down to ~ 60% for ey.

3. Use dropout after LSTM layer.

    - Dropout by randomly (binomially) setting 20% of activations to 0.
    - Use inverted dropout.
    - Much faster convergence.
    - Seems improvement for now. Still training.
    - Where to dropout?
    
4. N-way classification v.s. binary?

    - softmax to sigmoid
    - multitask
    - generative

5. SVM on LSTM output instead of logistic regression.

6. Transfer learning in current process?

7. Evaluate same speakers in Ey as in Breath.

8. Try less speakers with more samples
    
    - Generally improves the performance (of course!).
    
9. Mann–Whitney–Wilcoxon (MWW) Test

    - U test

#### Update since 1/23 meeting

**Dedicated to improving the performance of Ey.**

- Work on feature
    - DCT
    
    Purpose: the similar harmonic patterns in the Ey data confuses our classifier. These similar patterns are of low variations. We try to subtract the similar patterns from the Q-spectrogram by taking the DCT along the time axis and nulling the low frequency components.

    Result: Get worse. DCT takes out useful low-freq information.
    
    - normalize
    
    Purpose: center the data.
    
    Result: It exhibits less fluctuations at the begining stage of training. No improvement on the final performance.

    - log
    
    Purpose: just a hunch.
    
    Result: Get really bad because this messes the data scale.
    
    - augumentation with elastic transform
    
    Purpose: introduce random noise and generate more data to make the training more robust.
    
    Result: does not converge at all.
    
- Work on structure
    - small filters
    
    Small filters yield equivalent performance to large filters followed by smaller filters.
    
    Reason: equivalent reception fields.
    
    - additional lstm layer
    
    Take the sequential output from the first lstm layer and feed it into a second lstm layer.
    
    Reason: this might further capture the sequential information. 
    
    Result: Get worse, and really slow.
    
    - dropout at the lstm input
    
    Result: no difference.
    
    - dropout at the fully connected
    
    Results: (ongoing)
    
- Work on output
    - Open set identification
    Put the rest speakers in `Unknown` class.
    
    Result: ~ 60% accuracy. Training really slow.
    
- Verification, significance

    We want to analyze how exactly our classifier performs. On which speakers it does bad? Why does it perform bad? How to improve it?
    
    $H_0: $ same speaker 
    $H_1: $ not same speaker

    Test statistics:

    Confidence interval:

    Precision, recall, F-score:
    
    ROC, false positive, false negative:

#### Qual preparation

** 1/17 - 1/22 **
1. Forward and backward propagation
    - https://drive.google.com/drive/folders/0B9KsZNOgwwo_TXFxZU1GdmFQNG8?usp=sharing
    - Discussed forward and backward propagation, cross entropy, universal approximator, generative model, etc.
2. MFCC

** 1/23 - 1/28 **
1. Multilayer perceptron
2. Bias and variance
3. Linear algebra
4. DSP

Committee:
* Gary Overett
* Bhiksha (hard time!)
* Marios (nice)
* Stern

~~* Kumar (very strict! Failing machine)~~

* Byrun Yu (nice)

~~* Aarti Singh (not sure, good at machine learning, but nice?)~~

* Ashwin (fine)
* Soummya Kar (nice)
* Ian Lane (nice)

Abstract and 3 references

** Update 1/29 - 2/4 **

* Continue working on elastic transform

    * Tune the paramters and training. Current best: $\alpha=15$ for scale of displacement, $\sigma=2$ for Gaussian filter. For ey, different speakers
    ![elastic](./drafts/images/elastic.png)
    
    * Seems to work better than before: ~80% -> ~85% in accuracy for Ey.
    * 5 times data, significantly slower: one epoch for one night.
    * But error drops more per epoch: 50% error drops for the first epoch. 
    * Only performs bad on some certain speakers. Need to figure out why.
    
* 2d DCT
    * Perform 2d DCT, and then null the first 15 low frequency components in rows and cols.
    * Gives large training errors, and does not seem to converge.
    
* Using gradients
    * HOG features
        * The histogram of oriented gradients are extracted using 9 orientations, 8-by-8 pixels per cell, 3-by-3 cells per block. For ey, different speakers.
    ![hog_feat](./drafts/images/hog_feat.png)
    
    * Edge detection
        * with Sobel operator, for ey, different speakers
            ![edge_sobel](./drafts/images/edge_sobel.png)
    
* Finish the first draft of Qual paper

#### Courses

1. Statistical machine learning
2. Graphical models
3. Signal and systems
4. DSP

**Update 2/4 - 2/11**

1. Tried to find out the specific set of speakers that are often wrongly classified. Remove / substitute these speakers to improve classification accuracy.

2. Read papers and selected 3 reference papers.

3. Revised qual paper.

4. Prepared qual slides.

5. Got stuck for a while on PGM assignments.

## Update 2/19 - 2/25

1. Interspeech related
    1. Try out this similar framework
    
    http://yerevann.github.io/2016/06/26/combining-cnn-and-rnn-for-spoken-language-identification/
    
    This framework was suggested by Prof. Ming, and his lab proved that it is very effective.
    
        1. more convolutional layers (Alex-like)
        2. Batch normalize
        3. GRU
    2. Transfer learning
    
        1. Use out-of-set speakers to pre-train, like UBM?
        2. fix the pre-train layer, and just update the ~50 speakers.
        3. Extrame learning machine?
    3. Compare to GMM-UBM
    5. Check the frequency band of breath, and might want more bins in high frequencies; fuse const-q and dft.
    6. write up
2. Microfeature related
    1. Check codes
    2. Read literature
    3. Approaches
        1. ROI: region of interest
        2. Frequency of interest
        3. Template alphabet
        4. sparse model (***Richard G. Baraniuk***)
2. Qual related
    1. paper
    2. slides
    3. almost finished topics
3. Others
    1. Setup server and environment
    2. 702 project topic: Risk Analysis for Structured Prediction Algorithms

## Update 2/26 - 3/4

1. Interspeech
    1. write up https://www.sharelatex.com/project/58b60df66901f8d92ba0cce7
    2. run some results for paper
    3. Acc increases again (~90%)
2. Microfeature
    1. extract spectrograms and mfccs
    2. write code for computing gradient features
    3. write code for computing sparse models
    4. TIDIGITs (ming's scripts, parallel)    
* VOT

    RNN: `Automatic Measurement of Voice Onset Time and Prevoicing using Recurrent Neural Networks` Yossi Adi1, Joseph Keshet, Olga Dmitrieva, and Matt Goldrick    
3. Qual
    1. finished all topics
    2. revising slides 
4. Homework
    1. did a tedious machine learning homework